## Silverbox

Silverbox refers to one of the nonlinear system identification benchmarks on http://nonlinearbenchmark.org/#Silverbox. 
It is a simulation of a [Duffing oscillator](https://en.wikipedia.org/wiki/Duffing_equation), ocurring for instance in nonlinear spring pendulums.

State-space model description of the system:

$$\begin{align}
m \frac{d^2 x(t)}{dt^2} + v \frac{d x(t)}{dt} + a x(t) + b x^3(t) =&\ u(t) + w(t) \\
y(t) =&\ x(t) + e(t)
\end{align}$$

where
$$\begin{align}
m     =&\ \text{mass} \\
v     =&\ \text{viscous damping} \\
a     =&\ \text{linear stiffness} \\
b     =&\ \text{nonlinear stiffness} \\
y(t)    =&\ \text{observation (displacement)} \\
x(t)    =&\ \text{state (displacement)} \\
u(t)    =&\ \text{force} \\
e(t)    =&\ \text{measurement noise} \\
w(t)    =&\ \text{process noise}
\end{align}$$

### Solution steps

#### 1. Divide by leading coefficient m

We will reduce the equation to standard form by dividing through the mass coefficient.

$$\begin{align}
x''(t) + \frac{v}{m} x'(t) + \frac{a}{m} x(t) + \frac{b}{m} x^3(t) = \frac{1}{m} u(t) + \frac{1}{m}w(t) 
\end{align}$$

#### 2. Reduce to first-order differential equation

We make the following substitutions:

$$\begin{align} 
z_1(t) =&\ x(t) \\
z_2(t) =&\ x'(t) \, , 
\end{align}$$

which produces:

$$\begin{align}
z_1'(t) =&\ z_2(t) \\ 
z_2'(t) =&\ -\frac{v}{m} z_2(t) - \frac{a}{m} z_1(t) - \frac{b}{m} z_1^3(t) + \frac{1}{m} u(t) + \frac{1}{m}w(t)  \, .
\end{align}$$

We can re-write this into a matrix form:
$$\begin{align}
\underbrace{\begin{bmatrix} z_1'(t) \\ z_2'(t) \end{bmatrix}}_{\frac{d}{dt}z(t)} = \underbrace{\begin{bmatrix} 0 & 1 \\ -\frac{a}{m} -\frac{b}{m}z_1^2(t) & -\frac{v}{m} \end{bmatrix} \begin{bmatrix} z_1(t) \\ z_2(t) \end{bmatrix}}_{f(z, \theta)} + \underbrace{\begin{bmatrix} 0 \\ \frac{1}{m} \end{bmatrix}}_{B} u(t) + \begin{bmatrix} 0 \\ \frac{1}{m} \end{bmatrix} w(t)  \, .
\end{align}$$

where $\theta = (m,v,a,b)$.
 
#### 3. Discretize using Euler-Maruyama

We can perform an approximate discretization using Euler-Maruyama:

$$\begin{align}
\frac{d}{dt}z(t) \approx&\ \frac{z_{t+1} - z_{t}}{\Delta t} = f(z_t, \theta) + B u_t + B w_t \\
z_{t+1} - z_{t} =&\ f(z_t, \theta) \Delta t + B u_t \Delta t + B \Delta \beta_t \, .
\end{align}$$

where $\Delta \beta_t \sim \mathcal{N}(0, \tau^{-1} \Delta t)$ and $\Delta t = (t+1) - t = 1$. Plugging in $1$ for $\Delta t$ produces:

$$\begin{align}
z_{t+1} = g(z_t,\theta) + B u_t + B \Delta \beta_t \, .
\end{align}$$

where $g(z_t, \theta) = \begin{bmatrix} 1 & 1 \\ -\frac{a}{m} -\frac{b}{m} z_{1,t}^2 & -\frac{v}{m} + 1 \end{bmatrix} \begin{bmatrix} z_{1,t} \\ z_{2,t} \end{bmatrix}$

#### 4. Convert to Gaussian probabilities

Now I'm going to absorb $\frac{1}{m}$ into $w_t$ (using $\mathbb{V}[aX] = a^2\mathbb{V}[X]$):

$$\begin{align}
\mathbb{V}[\frac{1}{m} w_t] = \frac{1}{m^2} \mathbb{V}[w_t] = \frac{1}{m^2} \tau^{-1}
\end{align}$$

I will rename $\frac{1}{m^2} \tau^{-1}$ as $\gamma^{-1}$. Let $\tilde{\beta}_t \sim \mathcal{N}(0, \gamma^{-1})$. This yields:

$$\begin{align}
z_{t+1} = g(z_t,\theta) + B u_t + \tilde{\beta}_t \, .
\end{align}$$

We now have a discrete-time nonlinear state transition with white noise. We can therefore integrate out the noise variable to arrive at a Gaussian node function:

$$\begin{align}
z_{t+1} \sim&\ \mathcal{N}(g(z_{t},\theta) + B u_t, C(\gamma)) \\ 
y_t \sim&\ \mathcal{N}(c^{\top} z_t, \sigma^2) \, ,
\end{align}$$

for $C = \begin{bmatrix} 0 & 0 \\ 0 & \gamma^{-1} \end{bmatrix}$, $c = \begin{bmatrix} 1 & 0 \end{bmatrix}^{\top}$ and $e_t \sim \mathcal{N}(0, \sigma^2)$.

#### 6. Choose priors

I will first study a situation with known measurement noise (so $\sigma$ is fixed). I have chosen the following priors:

$$\begin{align}
m \sim&\ \Gamma(a^{0}_m, b^{0}_m) \\
v \sim&\ \mathcal{N}(m^{0}_v, v^{0}_v) \\
a \sim&\ \mathcal{N}(m^{0}_a, v^{0}_a) \\
b \sim&\ \mathcal{N}(m^{0}_b, v^{0}_b) \\
\gamma \sim&\ \Gamma(a^{0}_m, b^{0}_m)
\end{align}$$

Note: might substitute variables if other priors are necessary.

### Data

Let's first have a look at the data.

In [1]:
using Revise
using CSV
using DataFrames

In [2]:
using Plots

viz = false

false

In [3]:
# Read data from CSV file
df = CSV.read("../data/SNLS80mV.csv", ignoreemptylines=true)
df = select(df, [:V1, :V2])

# Shorthand
input = df[:,1]
output = df[:,2]

# Time horizon
T = size(df, 1);

In [4]:
if viz
    # Plot every n-th time-point to avoid figure size exploding
    n = 10
    p1 = Plots.scatter(1:n:T, output[1:n:T], color="black", label="output", markersize=2, size=(1600,800), xlabel="time (t)", ylabel="response")
    # Plots.savefig(p1, "viz/output_signal.png")
end

In [5]:
if viz
    p2 = Plots.scatter(1:n:T, input[1:n:T], color="blue", label="output", markersize=2, size=(1600,800), xlabel="time (t)", ylabel="control")
    # Plots.savefig(p2, "viz/input_signal.png")
end

## Estimating parameters via Bayesian filtering

Implementation with ForneyLab and AR node. The AR node is locally modified from the package LAR (LAR is in dev mode).

In [6]:
using ForneyLab
using ForneyLab: unsafeMean, unsafeCov, unsafeVar, unsafePrecision
using ProgressMeter

I will use the Nonlinear node to cope with a multivariate log-normal distribution.

In [19]:
# Start graph
graph = FactorGraph()

# Static parameters
@RV m ~ Gamma(placeholder(:a_m), placeholder(:a_b), id=:m)
@RV v ~ GaussianMeanPrecision(placeholder(:m_v), placeholder(:w_v), id=:v)
@RV a ~ GaussianMeanPrecision(placeholder(:m_a), placeholder(:w_a), id=:a)
@RV b ~ GaussianMeanPrecision(placeholder(:m_b), placeholder(:w_b), id=:b)
@RV γ ~ Gamma(placeholder(:a_γ), placeholder(:b_γ), id=:γ)

# Observation selection variable
c1 = [1. , 0.]
c2 = [0. , 1.]

# Measurement std
σ = 1e-3

# Control signal
# @RV u_t; placeholder(u_t, :u_t)
@RV u_t ~ GaussianMeanPrecision(placeholder(:m_u), [1e8], id=:u_t)

# State prior
@RV z_t ~ GaussianMeanPrecision(placeholder(:m_z, dims=(2,)), placeholder(:w_z, dims=(2, 2)), id=:z_t)

# Mean of state transition
g(z, m, v, a, b) = [1. 1.; -a/m-b/m*z[1]^2 -v/m+1]*z
@RV g_t ~ Nonlinear{Sampling}(z_t, m, v, a, b, g=g, dims=(2,), id=:g_t)

# Control term
h(u, m) = [0. u/m]
@RV Bu ~ Nonlinear{Sampling}(u_t, m, g=h, dims=(2,), id=:Bu)

# Process noise term
W(γ) = [1e-8 0.;0. γ]
@RV Wγ ~ Nonlinear{Sampling}(γ, g=W, dims=(2,2), id=:Wγ)

# State transition
@RV x_t ~ GaussianMeanPrecision(g_t + Bu, Wγ, id=:x_t)

# Observation likelihood
@RV y_t ~ GaussianMeanVariance(dot(c1, x_t), σ^2, id=:y_t)

# Placeholder for observation
placeholder(y_t, :y_t)

# Draw time-slice subgraph
ForneyLab.draw(graph)

# Infer an algorithm
q = PosteriorFactorization(z_t, x_t, m, v, a, b, γ, ids=[:z, :x, :m, :v, :a, :b, :γ])
algo = variationalAlgorithm(q, free_energy=false)
source_code = algorithmSourceCode(algo, free_energy=false)
eval(Meta.parse(source_code))
println(source_code)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 8180480010795597122 -->
 
 8180480010795597122 
 
 dot 
 dotproduct_1 
 
<!-- 2559181168569671608 -->
 
 2559181168569671608 
 
 𝒩 
 x_t 
 
<!-- 8180480010795597122--2559181168569671608 -->
 
 8180480010795597122--2559181168569671608 
 
 x_t 
 1 out 
 2 in1 
 
<!-- 2407816707187970498 -->
 
 2407816707187970498 
 
 clamp_2 
 
<!-- 8180480010795597122--2407816707187970498 -->
 
 8180480010795597122--2407816707187970498 
 
 clamp_2 
 1 out 
 3 in2 
 
<!-- 16354382981940956956 -->
 
 16354382981940956956 
 
 placeholder_y_t 
 
<!-- 3296751893289289201 -->
 
 3296751893289289201 
 
 𝒩 
 y_t 
 
<!-- 16354382981940956956--3296751893289289201 -->
 
 16354382981940956956--3296751893289289201 
 
 y_t 
 1 out 
 1 out 
 
<!-- 7023652099764321304 -->
 
 7023652099764321304 
 
 + 
 addition_1 
 
<!-- 1314080708164057121 -->
 
 1314080708164057121 
 
 Nonlinear{Sampling} 
 Bu 
 
<!-- 7023652099764321304--1314080708164057121 -->
 
 7023652099764321304--1314080708164057121 
 
 Bu 
 1 out 
 3 in2 
 
<!-- 17041747566097729173 -->
 
 17041747566097729173 
 
 Nonlinear{Sampling} 
 g_t 
 
<!-- 7023652099764321304--17041747566097729173 -->
 
 7023652099764321304--17041747566097729173 
 
 g_t 
 1 out 
 2 in1 
 
<!-- 11854745410413887685 -->
 
 11854745410413887685 
 
 clamp_1 
 
<!-- 3726536336519781498 -->
 
 3726536336519781498 
 
 𝒩 
 v 
 
<!-- 1430065086811169064 -->
 
 1430065086811169064 
 
 placeholder_m_v 
 
<!-- 3726536336519781498--1430065086811169064 -->
 
 3726536336519781498--1430065086811169064 
 
 m_v 
 1 out 
 2 m 
 
<!-- 14027027856995134977 -->
 
 14027027856995134977 
 
 placeholder_w_v 
 
<!-- 3726536336519781498--14027027856995134977 -->
 
 3726536336519781498--14027027856995134977 
 
 w_v 
 1 out 
 3 w 
 
<!-- 10458849146086039118 -->
 
 10458849146086039118 
 
 𝒩 
 z_t 
 
<!-- 15623124703789541529 -->
 
 15623124703789541529 
 
 placeholder_m_z 
 
<!-- 10458849146086039118--15623124703789541529 -->
 
 10458849146086039118--15623124703789541529 
 
 m_z 
 1 out 
 2 m 
 
<!-- 3265946386587607437 -->
 
 3265946386587607437 
 
 placeholder_w_z 
 
<!-- 10458849146086039118--3265946386587607437 -->
 
 10458849146086039118--3265946386587607437 
 
 w_z 
 1 out 
 3 w 
 
<!-- 77919373918617279 -->
 
 77919373918617279 
 
 placeholder_b_γ 
 
<!-- 10840213682961561763 -->
 
 10840213682961561763 
 
 placeholder_a_b 
 
<!-- 12189538914451881479 -->
 
 12189538914451881479 
 
 placeholder_a_γ 
 
<!-- 14207600093363129721 -->
 
 14207600093363129721 
 
 placeholder_w_a 
 
<!-- 6496073132368197507 -->
 
 6496073132368197507 
 
 𝒩 
 u_t 
 
<!-- 1314080708164057121--6496073132368197507 -->
 
 1314080708164057121--6496073132368197507 
 
 u_t 
 1 out 
 2 in1 
 
<!-- 15057023259668380854 -->
 
 15057023259668380854 
 
 Gam 
 m 
 
<!-- 15057023259668380854--10840213682961561763 -->
 
 15057023259668380854--10840213682961561763 
 
 a_b 
 1 out 
 3 b 
 
<!-- 16610601057842220523 -->
 
 16610601057842220523 
 
 placeholder_a_m 
 
<!-- 15057023259668380854--16610601057842220523 -->
 
 15057023259668380854--16610601057842220523 
 
 a_m 
 1 out 
 2 a 
 
<!-- 5775997774586781637 -->
 
 5775997774586781637 
 
 placeholder_w_b 
 
<!-- 12529388651561544085 -->
 
 12529388651561544085 
 
 placeholder_m_a 
 
<!-- 6496073132368197507--11854745410413887685 -->
 
 6496073132368197507--11854745410413887685 
 
 clamp_1 
 1 out 
 3 w 
 
<!-- 246751871228313409 -->
 
 246751871228313409 
 
 placeholder_m_u 
 
<!-- 6496073132368197507--246751871228313409 -->
 
 6496073132368197507--246751871228313409 
 
 m_u 
 1 out 
 2 m 
 
<!-- 10609661725843629190 -->
 
 10609661725843629190 
 
 = 
 equ_m_1 
 
<!-- 10609661725843629190--1314080708164057121 -->
 
 10609661725843629190--1314080708164057121 
 
 m 
 3 in2 
 3 3 
 


ArgumentError: ArgumentError: The input graph contains a loop around Interface 1 (out) of GaussianMeanPrecision v
.

The above factor graph complains about a loop in the Interface. In order to avoid the loop, I'm going to split the state distribution.

In [26]:
# Start graph
graph = FactorGraph()

# Static parameters
@RV μ ~ Gamma(placeholder(:a_μ), placeholder(:b_μ), id=:m)
@RV ν ~ GaussianMeanPrecision(placeholder(:m_ν), placeholder(:w_ν), id=:v)
@RV α ~ GaussianMeanPrecision(placeholder(:m_α), placeholder(:w_α), id=:a)
@RV β ~ GaussianMeanPrecision(placeholder(:m_β), placeholder(:w_β), id=:b)
@RV γ ~ Gamma(placeholder(:a_γ), placeholder(:b_γ), id=:γ)

# Observation selection variable
c1 = [1. , 0.]
c2 = [0. , 1.]

# Measurement std
σ = 1e-3

# Control signal
# @RV u_t; placeholder(u_t, :u_t)
@RV u_t ~ GaussianMeanPrecision(placeholder(:m_u), [1e8], id=:u_t)

# State prior
@RV z_1t ~ GaussianMeanPrecision(placeholder(:m_z1, dims=(1,)), placeholder(:w_z1, dims=(1, 1)), id=:z_1t)
@RV z_2t ~ GaussianMeanPrecision(placeholder(:m_z2, dims=(1,)), placeholder(:w_z2, dims=(1, 1)), id=:z_2t)

# Mean of state transition
g1(z, m, a, b) = -z*a/m -z^3*b/m
g2(z, m, v) = -z*v/m
@RV g_1t ~ Nonlinear{Unscented}(z_1t, μ, α, β, g=g1, dims=(1,), id=:g_1t)
@RV g_2t ~ Nonlinear{Unscented}(z_2t, μ, ν, g=g2, dims=(1,), id=:g_2t)

# Control term
h(u, m) = u/m
@RV h_t ~ Nonlinear{Sampling}(u_t, μ, g=h, dims=(1,), id=:h_t)

# State transition
@RV x_1t ~ GaussianMeanPrecision(z_1t + z_2t, 1e8, id=:x_1t)
@RV x_2t ~ GaussianMeanPrecision(g_1t + g_2t + h_t, γ, id=:x_2t)

# Observation likelihood
@RV y_t ~ GaussianMeanVariance(z_1t, σ^2, id=:y_t)

# Placeholder for observation
placeholder(y_t, :y_t)

# Draw time-slice subgraph
ForneyLab.draw(graph)

# Infer an algorithm
q = PosteriorFactorization(z_1t, z_2t, x_1t, x_2t, μ, ν, α, β, γ, ids=[:z1, :z2, :x1, :x2, :μ, :ν, :α, :β, :γ])
algo = variationalAlgorithm(q, free_energy=false)
source_code = algorithmSourceCode(algo, free_energy=false)
eval(Meta.parse(source_code));
# println(source_code)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 12229493869848081559 -->
 
 12229493869848081559 
 
 Nonlinear{Unscented} 
 g_1t 
 
<!-- 3867065851658292549 -->
 
 3867065851658292549 
 
 𝒩 
 a 
 
<!-- 12229493869848081559--3867065851658292549 -->
 
 12229493869848081559--3867065851658292549 
 
 α 
 1 out 
 4 in3 
 
<!-- 698947616580383622 -->
 
 698947616580383622 
 
 𝒩 
 b 
 
<!-- 12229493869848081559--698947616580383622 -->
 
 12229493869848081559--698947616580383622 
 
 β 
 1 out 
 5 in4 
 
<!-- 17638830013247466942 -->
 
 17638830013247466942 
 
 𝒩 
 u_t 
 
<!-- 2436446798156864647 -->
 
 2436446798156864647 
 
 placeholder_m_u 
 
<!-- 17638830013247466942--2436446798156864647 -->
 
 17638830013247466942--2436446798156864647 
 
 m_u 
 1 out 
 2 m 
 
<!-- 2555695461379245709 -->
 
 2555695461379245709 
 
 clamp_1 
 
<!-- 17638830013247466942--2555695461379245709 -->
 
 17638830013247466942--2555695461379245709 
 
 clamp_1 
 1 out 
 3 w 
 
<!-- 2618809110322648900 -->
 
 2618809110322648900 
 
 placeholder_w_α 
 
<!-- 9054761922124617742 -->
 
 9054761922124617742 
 
 placeholder_w_β 
 
<!-- 3867065851658292549--2618809110322648900 -->
 
 3867065851658292549--2618809110322648900 
 
 w_α 
 1 out 
 3 w 
 
<!-- 15393275532391395408 -->
 
 15393275532391395408 
 
 placeholder_m_α 
 
<!-- 3867065851658292549--15393275532391395408 -->
 
 3867065851658292549--15393275532391395408 
 
 m_α 
 1 out 
 2 m 
 
<!-- 13837077439294296001 -->
 
 13837077439294296001 
 
 placeholder_m_z1 
 
<!-- 15989130347120276884 -->
 
 15989130347120276884 
 
 Nonlinear{Unscented} 
 g_2t 
 
<!-- 6507066173952663589 -->
 
 6507066173952663589 
 
 𝒩 
 v 
 
<!-- 15989130347120276884--6507066173952663589 -->
 
 15989130347120276884--6507066173952663589 
 
 ν 
 1 out 
 4 in3 
 
<!-- 10614149573436662911 -->
 
 10614149573436662911 
 
 = 
 equ_μ_2 
 
<!-- 10614149573436662911--15989130347120276884 -->
 
 10614149573436662911--15989130347120276884 
 
 μ 
 3 in2 
 1 1 
 
<!-- 3913928961399614677 -->
 
 3913928961399614677 
 
 = 
 equ_μ_1 
 
<!-- 10614149573436662911--3913928961399614677 -->
 
 10614149573436662911--3913928961399614677 
 
 μ 
 3 3 
 2 2 
 
<!-- 10965907094843982499 -->
 
 10965907094843982499 
 
 Nonlinear{Sampling} 
 h_t 
 
<!-- 10614149573436662911--10965907094843982499 -->
 
 10614149573436662911--10965907094843982499 
 
 μ 
 3 in2 
 3 3 
 
<!-- 4499140630835774328 -->
 
 4499140630835774328 
 
 clamp_3 
 
<!-- 3913928961399614677--12229493869848081559 -->
 
 3913928961399614677--12229493869848081559 
 
 μ 
 3 in2 
 2 2 
 
<!-- 889510977677155438 -->
 
 889510977677155438 
 
 Gam 
 m 
 
<!-- 3913928961399614677--889510977677155438 -->
 
 3913928961399614677--889510977677155438 
 
 μ 
 1 out 
 1 1 
 
<!-- 5235893163834192269 -->
 
 5235893163834192269 
 
 + 
 addition_2 
 
<!-- 5235893163834192269--12229493869848081559 -->
 
 5235893163834192269--12229493869848081559 
 
 g_1t 
 1 out 
 2 in1 
 
<!-- 5235893163834192269--15989130347120276884 -->
 
 5235893163834192269--15989130347120276884 
 
 g_2t 
 1 out 
 3 in2 
 
<!-- 6976826957289701840 -->
 
 6976826957289701840 
 
 𝒩 
 y_t 
 
<!-- 6976826957289701840--4499140630835774328 -->
 
 6976826957289701840--4499140630835774328 
 
 clamp_3 
 1 out 
 3 v 
 
<!-- 10289571685634089918 -->
 
 10289571685634089918 
 
 𝒩 
 z_1t 
 
<!-- 10289571685634089918--13837077439294296001 -->
 
 10289571685634089918--13837077439294296001 
 
 m_z1 
 1 out 
 2 m 
 
<!-- 16979363722401976084 -->
 
 16979363722401976084 
 
 placeholder_w_z1 
 
<!-- 10289571685634089918--16979363722401976084 -->
 
 10289571685634089918--16979363722401976084 
 
 w_z1 
 1 out 
 3 w 
 
<!-- 9518874135818378569 -->
 
 9518874135818378569 
 
 = 
 equ_z_1t_1 
 
<!-- 9518874135818378569--12229493869848081559 -->
 
 9518874135818378569

ArgumentError: ArgumentError: The input graph contains a loop around Interface 1 (1) of Equality equ_μ_2
.

In [54]:
# Looking at only the first few timepoints
# T = 100
T = size(df, 1);

# Inference parameters
num_iterations = 10

# Initialize marginal distribution and observed data dictionaries
data = Dict()
marginals = Dict()

# Initialize arrays of parameterizations
params_x = (zeros(2,T+1), repeat(.1 .*float(eye(2)), outer=(1,1,T+1)))
params_θ = (zeros(2,T+1), repeat(.1 .*float(eye(2)), outer=(1,1,T+1)))
params_η = (zeros(1,T+1), 0.1 *ones(1,T+1))
params_ζ = (repeat([1e8 0.;0. 1.], outer=(1,1,T+1)), 2*ones(1,T+1))

# Start progress bar
p = Progress(T, 1, "At time ")

# Perform inference at each time-step
for t = 1:T

    # Update progress bar
    update!(p, t)

    # Initialize marginals
    marginals[:f_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_x[1][:,t], w=params_x[2][:,:,t])
    marginals[:z_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_x[1][:,t], w=params_x[2][:,:,t])
    marginals[:x_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_x[1][:,t], w=params_x[2][:,:,t])
    marginals[:θ] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=[0., 0.], w=[1. 0.; 0. 1.])
    marginals[:η] = ProbabilityDistribution(Univariate, GaussianMeanPrecision, m=0., w=1.)
    marginals[:eη] = ProbabilityDistribution(Univariate, GaussianMeanPrecision, m=0., w=1.)
    marginals[:ζ] = ProbabilityDistribution(MatrixVariate, Wishart, v=params_ζ[1][:,:,t], nu=params_ζ[2][1,t])
    
    data = Dict(:y_t => output[t],
                :m_u => input[t],
                :m_z => params_x[1][:,t],
                :W_z => params_x[2][:,:,t],
                :m_θ => params_θ[1][:,t],
                :W_θ => params_θ[2][:,:,t],
                :m_η => params_η[1][1,t],
                :w_η => params_η[2][1,t],
                :v_ζ => params_ζ[1][:,:,t],
                :n_ζ => params_ζ[2][1,t])

    # Iterate variational parameter updates
    for i = 1:num_iterations

#         stepz!(data, marginals)
        stepx!(data, marginals)
        stepθ!(data, marginals)
        stepη!(data, marginals)
        stepζ!(data, marginals)
#         steplθ!(data, marginals)
#         steplη!(data, marginals)
    end

    # Store current parameterizations of marginals
    params_x[1][:,t+1] = unsafeMean(marginals[:x_t])
    params_x[2][:,:,t+1] = marginals[:x_t].params[:w]
    params_θ[1][:,t+1] = unsafeMean(marginals[:θ])
    params_θ[2][:,:,t+1] = marginals[:θ].params[:w]
    params_η[1][1,t+1] = unsafeMean(marginals[:η])
    params_η[2][1,t+1] = marginals[:η].params[:w]
    params_ζ[1][:,:,t+1] = marginals[:ζ].params[:v]
    params_ζ[2][1,t+1] = marginals[:ζ].params[:nu]

end

MethodError: MethodError: no method matching ruleSPNonlinearUTInGX(::typeof(f), ::Int64, ::Message{GaussianMeanPrecision,Multivariate}, ::Message{GaussianMeanPrecision,Multivariate}, ::Message{GaussianMeanPrecision,Multivariate}, ::Message{GaussianMeanVariance,Univariate}, ::Message{GaussianMeanPrecision,Univariate})
Closest candidates are:
  ruleSPNonlinearUTInGX(::Function, ::Int64, ::Message{#s36,V} where #s36<:Gaussian, !Matched::Message{#s35,V} where #s35<:Gaussian...; alpha) where V<:ForneyLab.VariateType at /home/wmkouw/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_unscented.jl:214
  ruleSPNonlinearUTInGX(::Function, !Matched::Function, ::Message{#s40,V} where #s40<:Gaussian, !Matched::Union{Nothing, Message{#s39,V} where #s39<:Gaussian}...; alpha) where V<:ForneyLab.VariateType at /home/wmkouw/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_unscented.jl:183

### Visualize results

In [ ]:
viz = true

In [ ]:
# Extract mean of state marginals
estimated_states = params_x[1][1,2:end]

if viz
    # Plot every n-th time-point to avoid figure size exploding
    n = 10
    p1 = Plots.scatter(1:n:T, output[1:n:T], color="black", label="output", markersize=2, size=(1600,800), xlabel="time (t)", ylabel="response")
    Plots.plot!(1:n:T, estimated_states[1:n:T], color="red", linewidth=1, label="estimated")
#     Plots.savefig(p1, "viz/estimated_states01.png")
end

In [ ]:
# Extract mean of coefficient marginals
estimated_coeffs_1_mean = params_θ[1][1,2:end]
estimated_coeffs_1_std = sqrt.(inv.(params_θ[2][1,1,2:end]))
estimated_coeffs_2_mean = params_θ[1][2,2:end]
estimated_coeffs_2_std = sqrt.(inv.(params_θ[2][2,2,2:end]))

if viz
    # Plot both coefficients next to each other
    p2a = Plots.plot(1:n:T, estimated_coeffs_1_mean[1:n:T], ribbon=[estimated_coeffs_1_std[1:n:T], estimated_coeffs_1_std[1:n:T]], color="red", label="θ_1", xlabel="time (t)", ylim=[0., 0.8])
    p2b = Plots.plot(1:n:T, estimated_coeffs_2_mean[1:n:T], ribbon=[estimated_coeffs_2_std[1:n:T], estimated_coeffs_2_std[1:n:T]], color="blue", label="θ_2", xlabel="time (t)", ylim=[-0.1, 0.3])
    p2 = plot(p2a, p2b, size=(1600,600))
#     Plots.savefig(p2, "viz/estimated_coeffs.png")
end

In [ ]:
# Extract mean of control coefficient marginals
estimated_ccoeff_mean = exp.(params_η[1][1,2:end])
estimated_ccoeff_std = sqrt.(inv.(params_η[2][1,2:end]))

if viz
    # Plot both coefficients next to each other
    p3 = Plots.plot(1:n:T, estimated_ccoeff_mean[1:n:T], ribbon=[estimated_ccoeff_std[1:n:T], estimated_ccoeff_std[1:n:T]], color="blue", label="η", xlabel="time (t)", size=(800,600), ylim=[0.0, 0.25])
#     Plots.savefig(p3, "viz/estimated_ccoeff.png")
end

In [ ]:
# Extract mean of process precision marginals
estimated_pnoise_mean = params_τ[1][1,2:end] ./ params_τ[2][1,2:end]
estimated_pnoise_std = sqrt.(params_τ[1][1,2:end] ./ params_τ[2][1,2:end].^2)

if viz
    # Plot both coefficients next to each other
    p4 = Plots.plot(1:n:T, estimated_pnoise_mean[1:n:T], ribbon=[estimated_pnoise_std[1:n:T], estimated_pnoise_std[1:n:T]],color="blue", label="τ", xlabel="time (t)")
#     Plots.savefig(p4, "viz/estimated_pnoise.png")
end